Загружаем необходимые библиотеки

In [3]:
from openpyxl import load_workbook
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import Normalizer
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier


Так как библиотека Pandas не позволяет получить информацию непосредственно о цвете текста в файле Excel, в
ыполним разделение на классы перед выполнением программы

In [106]:
book = load_workbook('V6RS41at_11.05.2014ТЗЧ_или_СС.xlsx')
sheet = book.active
classes = []

for row in range(2,  sheet.max_row):
    red_cell_in_row_ones= False
    red_cell_in_row = False

    for column in range(1, sheet.max_column):
        cell = sheet[row][column]
        row_value = row - 1
        while cell.value is None:
            cell = sheet[row_value][column]
            row_value -= 1
        if cell.font.color.value == 'FFFF0000' and red_cell_in_row_ones == True:
            red_cell_in_row = True
            break
        elif cell.font.color.value == 'FFFF0000':
            red_cell_in_row_ones = True

    if red_cell_in_row:
        if classes[-1] == 0:
            classes.append(2)
        elif classes[-1] == 1:
            classes.append(1)
        elif classes[-3:] == [2, 2, 2]:
            classes = classes[:-3] + [1, 1, 1, 1]
        elif classes[-1] == 2:
            classes.append(2)
    else:
        classes.append(0)

print(classes)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 

Считываем данные

In [7]:
data = pd.read_excel('V6RS41at_11.05.2014ТЗЧ_или_СС.xlsx')
data

,Unnamed: 0,"Ubs,V","Ibs,A","Isun,A","RKPT,b","Sst0x,b","Sst1x,b","Ipt0,A","Ipt1,A","Ipt2,A",...,"SPrm2,V","VP,d","ShKom,d","Kmnd,d","God,h","Mes,h","Den,h","Chas,h","Min,h",Sec
0,2014-05-11 00:00:30,14.17,0.24,0.00,1000000.0,110.0,0.0,0.0,0.07,0.07,...,0.98,0.0,68.0,2010.0,14,5,11,0,0,30
1,2014-05-11 00:01:30,14.17,0.21,0.00,1000000.0,110.0,0.0,0.0,0.07,0.09,...,0.98,0.0,68.0,2010.0,14,5,11,0,1,30
2,2014-05-11 00:02:30,14.17,0.27,0.00,1000000.0,110.0,0.0,0.0,0.09,0.09,...,0.98,0.0,68.0,2010.0,14,5,11,0,2,30
3,2014-05-11 00:03:30,14.17,0.27,0.00,1000000.0,110.0,0.0,0.0,0.07,0.07,...,0.99,0.0,68.0,2010.0,14,5,11,0,3,30
4,2014-05-11 00:04:30,14.17,0.27,0.00,1000000.0,110.0,0.0,0.0,0.07,0.07,...,0.99,0.0,68.0,2010.0,14,5,11,0,4,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,2014-05-11 22:44:08,14.17,0.24,0.00,1000000.0,110.0,0.0,0.0,0.07,0.07,...,1.04,0.0,123.0,2615.0,14,5,11,22,44,7
1362,2014-05-11 22:45:08,14.17,0.24,0.00,1000000.0,110.0,0.0,0.0,0.04,0.07,...,1.10,0.0,123.0,2615.0,14,5,11,22,45,8
1363,2014-05-11 22:46:08,14.30,0.21,1.09,1000000.0,110.0,0.0,0.0,0.09,0.07,...,1.39,0.0,123.0,2615.0,14,5,11,22,46,8
1364,2014-05-11 22:47:08,14.50,0.24,1.31,1000000.0,110.0,0.0,0.0,0.09,0.07,...,1.18,0.0,123.0,2615.0,14,5,11,22,47,8


Удаляем необходимые столбцы

In [8]:
signs_to_remove = ['RKPT,b','Sst0x,b','Sst1x,b','Ipt0,A', 'Mag0F', 'Mag10', 'Mag11', 'Mag12', 'Mag13',
                   'Mag14','Mag15','Mag16','Isp1,A','Isp2,A','Isp3,A','Isp4,A','Isp5,A','Isp6,A','Isp7,A',
                   'Isp8,A','Mag07','Mag08','Mag09','Mag0A','Mag0B','Mag0C','Mag0D','Mag0E','Mag03','Mag04',
                   'Mag05','Mag06','TDS11,C','TDS12,C','TDS14,C','TDS15,C','TDS17,C','TDS18,C','TDS20,C',
                   'TDS21,C','TDS23,C','iTMO,s','iTMD,s','U-chCAN', 'SSoTR,b','STDS12,b','STDS34,b','res0',
                   'U-MAG,o','U-KPT1,o','U-KPT2,o','U-InfGBV','U-GBV,o','U-InfNAP','TmUpr,b','RKwit,b','Rmorz,b',
                   'IntK_TM', 'RPrd1,b','RPrm1,b','RPrd2,b','RPrm2,b','FPrd1,V','RPrd1,V','FPrd2,V','RPrd2,V','SPrm1,V',
                   'SPrm2,V','VP,d','ShKom,d','Kmnd,d','God,h','Mes,h','Den,h','Chas,h', 'Min,h', 'Sec']
cleared_data = data.copy()
cleared_data.drop(data.columns[[0]], axis=1, inplace=True)
for sign in signs_to_remove:
    cleared_data.drop(columns=[sign], axis= 1, inplace=True)
cleared_data

,"Ubs,V","Ibs,A","Isun,A","Ipt1,A","Ipt2,A","Ipt3,A","Ipt4,A","Ipt5,A","Ipt6,A","Ipt7,A",...,"TDS6,C","TDS7,C","TDS8,C","TDS9,C","TKpt,C","TGbv,C","TNap,C","TPrd2,C","TPrd1,C","TDS24,C"
0,14.17,0.24,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,19.0,19.0,27.0,20.0,20.0,20.0,20.0,23.0
1,14.17,0.21,0.00,0.07,0.09,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,19.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0
2,14.17,0.27,0.00,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,19.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0
3,14.17,0.27,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,18.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0
4,14.17,0.27,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,18.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,14.17,0.24,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,19.0,19.0,27.0,22.0,22.0,21.0,21.0,23.0
1362,14.17,0.24,0.00,0.04,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,19.0,19.0,27.0,22.0,22.0,21.0,21.0,23.0
1363,14.30,0.21,1.09,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,19.0,19.0,27.0,22.0,22.0,21.0,21.0,23.0
1364,14.50,0.24,1.31,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,17.0,18.0,19.0,19.0,27.0,22.0,21.0,21.0,21.0,23.0


Заполняем пустые значения

In [9]:
data_filled = cleared_data.copy()
data_filled = data_filled.fillna(method='ffill')
data_filled

,"Ubs,V","Ibs,A","Isun,A","Ipt1,A","Ipt2,A","Ipt3,A","Ipt4,A","Ipt5,A","Ipt6,A","Ipt7,A",...,"TDS6,C","TDS7,C","TDS8,C","TDS9,C","TKpt,C","TGbv,C","TNap,C","TPrd2,C","TPrd1,C","TDS24,C"
0,14.17,0.24,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,19.0,19.0,27.0,20.0,20.0,20.0,20.0,23.0
1,14.17,0.21,0.00,0.07,0.09,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,19.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0
2,14.17,0.27,0.00,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,19.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0
3,14.17,0.27,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,18.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0
4,14.17,0.27,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,18.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,14.17,0.24,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,19.0,19.0,27.0,22.0,22.0,21.0,21.0,23.0
1362,14.17,0.24,0.00,0.04,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,19.0,19.0,27.0,22.0,22.0,21.0,21.0,23.0
1363,14.30,0.21,1.09,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,19.0,19.0,27.0,22.0,22.0,21.0,21.0,23.0
1364,14.50,0.24,1.31,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,17.0,18.0,19.0,19.0,27.0,22.0,21.0,21.0,21.0,23.0


Выполним нормализацию данных

In [10]:
# Преобразовываем исходный набор
scaler = Normalizer().fit(data_filled)
rescaled_data = pd.DataFrame(scaler.fit_transform(data_filled))
rescaled_data

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,0.154960,0.002625,0.000000,0.000766,0.000766,0.0,0.0,0.0,0.0,0.0,...,0.196844,0.196844,0.207779,0.207779,0.295265,0.218715,0.218715,0.218715,0.218715,0.251522
1,0.156112,0.002314,0.000000,0.000771,0.000992,0.0,0.0,0.0,0.0,0.0,...,0.198308,0.198308,0.209325,0.198308,0.297461,0.220342,0.220342,0.220342,0.220342,0.253393
2,0.157397,0.002999,0.000000,0.001000,0.001000,0.0,0.0,0.0,0.0,0.0,...,0.199940,0.199940,0.211048,0.199940,0.299910,0.222155,0.222155,0.222155,0.222155,0.255479
3,0.158209,0.003015,0.000000,0.000782,0.000782,0.0,0.0,0.0,0.0,0.0,...,0.200972,0.200972,0.200972,0.200972,0.301457,0.223302,0.223302,0.223302,0.223302,0.256797
4,0.159933,0.003047,0.000000,0.000790,0.000790,0.0,0.0,0.0,0.0,0.0,...,0.203161,0.203161,0.203161,0.203161,0.304742,0.225735,0.225735,0.225735,0.225735,0.259595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,0.154407,0.002615,0.000000,0.000763,0.000763,0.0,0.0,0.0,0.0,0.0,...,0.196141,0.207038,0.207038,0.207038,0.294212,0.239728,0.239728,0.228831,0.228831,0.250625
1362,0.155248,0.002629,0.000000,0.000438,0.000767,0.0,0.0,0.0,0.0,0.0,...,0.197209,0.208166,0.208166,0.208166,0.295814,0.241034,0.241034,0.230078,0.230078,0.251990
1363,0.158549,0.002328,0.012085,0.000998,0.000776,0.0,0.0,0.0,0.0,0.0,...,0.199573,0.210660,0.210660,0.210660,0.299359,0.243922,0.243922,0.232835,0.232835,0.255009
1364,0.165501,0.002739,0.014952,0.001027,0.000799,0.0,0.0,0.0,0.0,0.0,...,0.194035,0.205449,0.216863,0.216863,0.308173,0.251104,0.239690,0.239690,0.239690,0.262518


Присоединяем столбец классов

In [11]:
rescaled_data_with_classes = rescaled_data.copy()
data_with_classes = data_filled.copy()

rescaled_data_with_classes.insert(loc = rescaled_data_with_classes.shape[1], column = 'Class', value = classes)
data_with_classes.insert(loc = data_with_classes.shape[1], column = 'Class', value = classes)
data_with_classes

,"Ubs,V","Ibs,A","Isun,A","Ipt1,A","Ipt2,A","Ipt3,A","Ipt4,A","Ipt5,A","Ipt6,A","Ipt7,A",...,"TDS7,C","TDS8,C","TDS9,C","TKpt,C","TGbv,C","TNap,C","TPrd2,C","TPrd1,C","TDS24,C",Class
0,14.17,0.24,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,19.0,27.0,20.0,20.0,20.0,20.0,23.0,0
1,14.17,0.21,0.00,0.07,0.09,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0,0
2,14.17,0.27,0.00,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0,0
3,14.17,0.27,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0,0
4,14.17,0.27,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,18.0,18.0,27.0,20.0,20.0,20.0,20.0,23.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,14.17,0.24,0.00,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,19.0,19.0,19.0,27.0,22.0,22.0,21.0,21.0,23.0,0
1362,14.17,0.24,0.00,0.04,0.07,0.0,0.0,0.0,0.0,0.0,...,19.0,19.0,19.0,27.0,22.0,22.0,21.0,21.0,23.0,0
1363,14.30,0.21,1.09,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,19.0,19.0,19.0,27.0,22.0,22.0,21.0,21.0,23.0,0
1364,14.50,0.24,1.31,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,18.0,19.0,19.0,27.0,22.0,21.0,21.0,21.0,23.0,0


Проверим, заполнились ли пустые значение

In [12]:
data_with_classes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366 entries, 0 to 1365
Data columns (total 50 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ubs,V    1366 non-null   float64
 1   Ibs,A    1366 non-null   float64
 2   Isun,A   1366 non-null   float64
 3   Ipt1,A   1366 non-null   float64
 4   Ipt2,A   1366 non-null   float64
 5   Ipt3,A   1366 non-null   float64
 6   Ipt4,A   1366 non-null   float64
 7   Ipt5,A   1366 non-null   float64
 8   Ipt6,A   1366 non-null   float64
 9   Ipt7,A   1366 non-null   float64
 10  Ipt10,A  1366 non-null   float64
 11  Ipt11,A  1366 non-null   float64
 12  Ipt12,A  1366 non-null   float64
 13  Ipt13,A  1366 non-null   float64
 14  Ipt14,A  1366 non-null   float64
 15  Ipt15,A  1366 non-null   float64
 16  Ipt16,A  1366 non-null   float64
 17  Ipt17,A  1366 non-null   float64
 18  TR1,C    1366 non-null   float64
 19  TR2,C    1366 non-null   float64
 20  TR3,C    1366 non-null   float64
 21  TR4,C    1366 

Проверим, сколько образцов относятся к каждому классу

In [13]:
class_counts = data_with_classes['Class'].value_counts()
class_counts

0    1197
1     123
2      46
Name: Class, dtype: int64

Разделим набор на обучащую и тестовую выборки

In [14]:
test_size = 0.2
seed = 7

data_X = data_with_classes.iloc[:, :-1]
data_Y = data_with_classes['Class']

data_X_train, data_X_test, data_Y_train, data_Y_test = train_test_split(data_X, data_Y, test_size=test_size, random_state=seed)

rescaled_data_X = rescaled_data_with_classes.iloc[:, :-1]
rescaled_data_Y = rescaled_data_with_classes['Class']

rescaled_data_X_train, rescaled_data_X_test, rescaled_data_Y_train, rescaled_data_Y_test = train_test_split(rescaled_data_X, rescaled_data_Y, test_size=test_size, random_state=seed)


datasets = [{'X_train': data_X_train, 'X_test': data_X_test, 'Y_train': data_Y_train, 'Y_test': data_Y_test, 'name': 'original_data'},
            {'X_train': rescaled_data_X_train, 'X_test': rescaled_data_X_test, 'Y_train': rescaled_data_Y_train, 'Y_test': rescaled_data_Y_test, 'name': 'original_rescaled_data'}]

Реализация LSTM нейросетевой модели

In [15]:

number_of_neurons = 32


for dataset in datasets:
    print(f'Analyse for {dataset.get("name")}')
    X_train = dataset.get('X_train')
    Y_train = to_categorical(dataset.get('Y_train'), num_classes=3)

    X_test = dataset.get('X_test')
    Y_test = to_categorical(dataset.get('Y_test'), num_classes=3)
    
    print(f'Classification for train data')

    model = Sequential()
    model.add(Reshape((49, 1), input_shape=(49,)))
    model.add(LSTM(number_of_neurons, recurrent_dropout=0.2, activation='relu',return_sequences=True))
    model.add(LSTM(number_of_neurons, recurrent_dropout=0.2, activation='relu',return_sequences=True))
    model.add(LSTM(number_of_neurons, recurrent_dropout=0.2, activation='relu'))

    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # добавляем обратный вызов EarlyStopping
    early_stop = EarlyStopping(monitor='val_loss', patience=3)

    history = model.fit(X_train,
                        Y_train,
                        epochs=10,
                        batch_size=32,
                        validation_split=0.1,
                        callbacks=[early_stop],
                        ) 

    # выводим номер лучшей эпохи
    best_epoch = np.argmin(history.history['val_loss']) + 1
    print(f"Best epoch: {best_epoch}")
    
    # получаем прогнозы модели на тестовых данных
    Y_pred = np.argmax(model.predict(X_test), axis=-1)
    
    # преобразуем метки классов и предсказания в одномерные массивы
    Y_test_flat = np.argmax(Y_test, axis=1)
    Y_pred_flat = Y_pred.flatten()
      
    print(f'Classification for test data')

    # выводим отчет о классификации
    print(classification_report(Y_test_flat, Y_pred_flat, zero_division=1))

Analyse for original_data
Classification for train data
Epoch 1/10
31/31 [==============================] - 17s 249ms/step - loss: 0.4004 - accuracy: 0.9134 - val_loss: 0.1699 - val_accuracy: 0.9545
Epoch 2/10
31/31 [==============================] - 6s 180ms/step - loss: 0.2669 - accuracy: 0.9481 - val_loss: 0.1491 - val_accuracy: 0.9636
Epoch 3/10
31/31 [==============================] - 4s 142ms/step - loss: 0.1798 - accuracy: 0.9501 - val_loss: 0.1326 - val_accuracy: 0.9455
Epoch 4/10
31/31 [==============================] - 5s 158ms/step - loss: 0.1882 - accuracy: 0.9552 - val_loss: 0.1529 - val_accuracy: 0.9636
Epoch 5/10
31/31 [==============================] - 5s 156ms/step - loss: 0.1410 - accuracy: 0.9644 - val_loss: 0.1278 - val_accuracy: 0.9636
Epoch 6/10
31/31 [==============================] - 4s 138ms/step - loss: 0.1837 - accuracy: 0.9582 - val_loss: 0.1477 - val_accuracy: 0.9364
Epoch 7/10
31/31 [==============================] - 5s 179ms/step - loss: 0.1366 - accuracy

Определим гиперпараметры для бэггинга

In [16]:
param_grid = {'n_estimators': [100],
              'max_samples': [0.5,0.75,0.99],
              'estimator__max_depth': [5,10,15]}

Реализация Бэггинга на основе CART

In [17]:


for dataset in datasets:
    print(f'Analyse for {dataset.get("name")}')
    tree_class = DecisionTreeClassifier()
    bagging_model = BaggingClassifier(tree_class)

    grid_search = GridSearchCV(bagging_model, param_grid)
    grid_search.fit(dataset.get('X_train'), dataset.get('Y_train'))

    best_params_for_data = grid_search.best_params_
    print(f"Best hyper parameters for bagging: {best_params_for_data}")

    tree_model_train_data = DecisionTreeClassifier(max_depth=best_params_for_data.get('max_depth'),
                                                random_state=42)
    bagging_model_train_data = BaggingClassifier(tree_model_train_data,
                                                n_estimators=best_params_for_data.get('n_estimators'),
                                                max_samples=best_params_for_data.get('max_samples'))
    bagging_model_train_data.fit(dataset.get('X_train'), dataset.get('Y_train'))

    bagging_model_train_data.fit(dataset.get('X_train'), dataset.get('Y_train'))
    train_Y_pred = bagging_model_train_data.predict(dataset.get('X_train'))
    print('Classification report for train data')
    print(classification_report(dataset.get('Y_train'), train_Y_pred))

    tree_model_test_data = DecisionTreeClassifier(max_depth=best_params_for_data.get('max_depth'),
                                                random_state=42)
    bagging_model_test_data = BaggingClassifier(tree_model_test_data,
                                                n_estimators=best_params_for_data.get('n_estimators'),
                                                max_samples=best_params_for_data.get('max_samples'))
    bagging_model_test_data.fit(dataset.get('X_train'), dataset.get('Y_train'))
    test_Y_pred = bagging_model_test_data.predict(dataset.get('X_test'))
    print('Classification report for test data')
    print(classification_report(dataset.get('Y_test'), test_Y_pred))

Analyse for original_data
Best hyper parameters for bagging: {'estimator__max_depth': 15, 'max_samples': 0.75, 'n_estimators': 100}
Classification report for train data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       955
           1       1.00      1.00      1.00       100
           2       1.00      0.95      0.97        37

    accuracy                           1.00      1092
   macro avg       1.00      0.98      0.99      1092
weighted avg       1.00      1.00      1.00      1092

Classification report for test data
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       242
           1       0.87      0.87      0.87        23
           2       0.83      0.56      0.67         9

    accuracy                           0.97       274
   macro avg       0.90      0.81      0.84       274
weighted avg       0.97      0.97      0.97       274

Analyse for original_rescaled_dat

В случае реализации нейросетевой LSTM модели лучшие результаты показал непреобразованный набор данных, в случае с бэггингом на основе CART мы наблюдаем  ситуацию, где лучший результат показала классификаия непреобразованный тренировочный набор. Обели модели показывают плохие оценки для второго класса на тестовом наборе данных, это может быть связанно с их количеством на фоне остальных данных.По результатам исследования, лучшей моделью является модель бэггинга на основе CART на непреобразованном наборе данных.